In [43]:
#write config file
import json
json_path = "C:/Users/nchong/OneDrive - Intel Corporation/Documents/Debug Similarity Analytics and Bucketization Framework/General/"
path = "C:/Users/nchong/OneDrive - Intel Corporation/Documents/Debug Similarity Analytics and Bucketization Framework/General/Sample json output/HSD ES Raw Data/team1/"

config = {
    "DataLoading": {"path": path, "start_date": None,"stop_date":None},
    "DataPreprocessing":{
    "df_manipulation": {"col_selection":["id","description"],"keep": None,"subset":None},    
    "word_contractions": {"enable": True},
    "lowercase": {"enable": True},
    "remove_htmltag_url": {"enable":True},
    "remove_irrchar_punc":{"enable":True,"char":None},
    "remove_num":{"enable":True},
    "remove_multwhitespace":{"enable":True},
    "remove_stopwords":{"enable":True,"extra_sw":None,"remove_sw": None},
    "remove_freqwords":{"enable":True,"n":10},
    "remove_rarewords":{"enable":True,"n":10},
    "custom_taxo":{"enable":True,"remove_taxo":["gio","fields","test"],"include_taxo":["test suite execution","kpi metric"]},
    "stem_words":{"enable":False,"stemmer_type":None},
    "lemmatize_words":{"enable":True,"lemma_type":None}
    },
    "UnsupervisedLearning":{
        "kmeans_clustering":{"enable":True,"top_n_terms":10,"ngram_range":None,"fe_type":None,"n_clusters":None,"max_n_clusters":10}
        "lda":{"enable":True,"n_components":5,"top_n_terms":10,"ngram_range"=None}
        "nmf"{"enable":True,"n_components":5,"top_n_terms":10,"ngram_range"=None,"fe_type":None}
    },:
    "SupervisedLearning":{
        "supervised_lng":{"enable":True,"test_size":0.3,"ngram_range":None,"fe_type":None,"model_type":None,"ascend":None,"save_path":None}
        "deep_lng":{"enable":True,"test_size":0.3,"ngram_range":None,"fe_type":None,"hidden_layer_sizes":(5,5),"activation":None,"solver":None,"learning_rate":None,"max_iter":None,"ascend":None,"save_path":None}
    },
    "SimilarityMetrics":{
        "cosinesimilarity"{"enable":True,"threshold":None,"total_rows":10,"base_row":None,"ngram_range":None,"fe_type":None,"ascending":None}
        "jaccard_similarity"{"enable":True,"threshold":None,"total_rows":10,"base_row":None,"ascending":None}
    }

}

with open(json_path+'config.json', 'w') as f:
    json.dump(config,f,indent=8)

In [63]:
out_path = json_path

In [61]:
#read config file and call the other functions
def main(json_path,out_path):
    
    import json     
    import pandas as pd
    
    with open(json_path+'config.json') as config_file:
        data = json.load(config_file)
    
    #---------DATA LOADING----------#
    dl = data["DataLoading"]
    path,start_date,stop_date = dl['path'],dl['start_date'],dl['stop_date']  
    df = data_loading(path=path,start_date=start_date,stop_date=stop_date)

    #---------DATA PREPROCESSING----------# 
    #df_manipulation
    dm = data["DataPreprocessing"]["df_manipulation"]
    col_selection,keep,subset = dm['col_selection'],dm['keep'],dm['subset']  
    df = df_manipulation(df,col_selection=col_selection,keep=keep,subset=subset)
    df_all = df.copy() #id, raw text -> data preprocessing final file
    df_out = df.copy() #id, raw text -> ml output
    df = df.drop("id",axis=1) #raw text
    
    #word_contractions
    wordcont = data["DataPreprocessing"]["word_contractions"]["enable"]    
    if wordcont:
        df = word_contractions(df)
        df_all = pd.concat([df_all,df],axis=1) #raw text, raw text after contractions
            
    #lowercase
    lower = data["DataPreprocessing"]["lowercase"]["enable"]      
    if lower:
        df = lowercase(df)
        df_all = pd.concat([df_all,df],axis=1)

            
   #Remove html tag and url
    tagrem = data["DataPreprocessing"]["remove_htmltag_url"]["enable"] 
    if tagrem:
        df = remove_htmltag_url(df)
        df_all = pd.concat([df_all,df],axis=1)

    #Remove irrelevant characters and punctuation
    pc = data["DataPreprocessing"]["remove_irrchar_punc"]
    puncrem,char = pc["enable"],pc["char"] 
    if puncrem:
        df = remove_irrchar_punc(df,char=char)
        df_all = pd.concat([df_all,df],axis=1)
        
    #Remove numbers 
    numrem = data["DataPreprocessing"]["remove_num"]["enable"]
    if numrem:
        df = remove_num(df)
        df_all = pd.concat([df_all,df],axis=1)

    # Remove multiple whitespace
    wsrem = data["DataPreprocessing"]["remove_multwhitespace"]["enable"]
    if wsrem:
        df = remove_multwhitespace(df)
        df_all = pd.concat([df_all,df],axis=1)

    # Remove stopwords
    stoprem = data["DataPreprocessing"]["remove_stopwords"]["enable"]
    if stoprem:
        df = remove_stopwords(df,extra_sw=None,remove_sw=None)
        df_all = pd.concat([df_all,df],axis=1)
        
    # Remove frequent words
    fw = data["DataPreprocessing"]["remove_freqwords"]
    freqrem,n = fw["enable"],fw["n"]
    if freqrem:
        df = remove_freqwords(df,n)
        df_all = pd.concat([df_all,df],axis=1)
    
    # Remove rare words
    rw = data["DataPreprocessing"]["remove_rarewords"]
    rarerem,n = rw["enable"],rw["n"]    
    if rarerem:
        df = remove_rarewords(df,n)
        df_all = pd.concat([df_all,df],axis=1)
        
    # Custom taxonomy
    ct = data["DataPreprocessing"]["custom_taxo"]
    taxo,remove_taxo,include_taxo = ct["enable"], ct["remove_taxo"], ct["include_taxo"]
    if taxo:
        df = custom_taxo(df,remove_taxo,include_taxo)
        df_all = pd.concat([df_all,df],axis=1)     
        
    # Stemming
    st = data["DataPreprocessing"]["stem_words"]
    stem,stemmer_type = st["enable"],st["stemmer_type"]     
    if stem:
        df = stem_words(df,stemmer_type)
        df_all = pd.concat([df_all,df],axis=1)            
    
    #Lemmatization
    lem = data["DataPreprocessing"]["lemmatize_words"]
    lemma,lemma_type = lem["enable"], lem["lemma_type"]      
    if lemma:
        df = lemmatize_words(df,lemma_type)
        df_all = pd.concat([df_all,df],axis=1)
    
    display(df_all)    
    df_all.to_csv(out_path+"preprocessed_text.csv",index=False) #save after data preprocessing   
    
    #---------------ML module---------------# 
    #Unsupervised Learning
    #k-means clustering 
    km= data["UnsupervisedLearning"]["kmeans_clustering"]
    kmeans = km["enable"]
    if kmeans:
        top_n_terms,ngram_range,fe_type,n_clusters,max_n_clusters= km["top_n_terms"],km["ngram_range"],km["fe_type"],km["n_clusters"],km["max_n_clusters"]        
        df_out["cluster"]=kmeans_clustering(column=df,top_n_terms=top_n_terms,ngram_range=ngram_range,fe_type=fe_type,n_clusters=n_clusters,max_n_clusters=max_n_clusters)
        display(df_out)        
        df_out.to_csv(out_path+"kmeans_output.csv",index=False)      
        
    #LDA
    lda_m= data["UnsupervisedLearning"]["lda"]
    LatentDirichletAllocation = lda_m["enable"]
    if LatentDirichletAllocation:
        n_components,top_n_terms,ngram_range= lda_m["n_components"],lda_m["top_n_terms"],lda_m["ngram_range"]       
        df_out["topic"]=lda(column=df,n_components=n_components,top_n_terms=top_n_terms,ngram_range=ngram_range)
        display(df_out)        
        df_out.to_csv(out_path+"LatentDirichletAllocation_output.csv",index=False)       
        
    #NMF Factorization
    nmf_m= data["UnsupervisedLearning"]["nmf"]
    NonNegativeMatrixFactorization = nmf_m["enable"]
    if NonNegativeMatrixFactorization:
        top_n_terms,ngram_range,fe_type,n_clusters,max_n_clusters= nmf_m["n_components"],nmf_m["top_n_terms"],nmf_m["fe_type"],nmf_m["ngram_range"]
        df_out["topic"]=nmf(column=df,n_components=n_components,top_n_terms=top_n_terms,fe_type=fe_type,ngram_range=ngram_range)
        display(df_out)        
        df_out.to_csv(out_path+"NonNegativeMatrixFactorization_output.csv",index=False) 
        
    #Supervised Learning
    #Machine Learning
    sl= data["SupervisedLearning"]["supervised_lng"]
    SupervisedLearning = sl["enable"]
    if SupervisedLearning:
        test_size,ngram_range,fe_type,model_type,ascend,save_path= sl["test_size"],sl["ngram_range"],sl["fe_type"],sl["model_type"],sl["ascend"],sl["save_path"]
        supervised_lng(X=df,y=df,test_size=test_size,ngram_range=ngram_range,fe_type=fe_type,model_type=model_type,ascend=ascend,save_path=save_path)
    
    #Deep Learning
    dl= data["SupervisedLearning"]["deep_lng"]
    DeepLearning = dl["enable"]
    if DeepLearning:
        test_size,ngram_range,fe_type,hidden_layer_sizes,activation,solver,learning_rate,max_iter,ascend,save_path= dl["test_size"],dl["ngram_range"],dl["fe_type"],dl["hidden_layer_sizes"],dl["activation"],dl["solver"],dl["learning_rate"],dl["max_iter"],dl["ascend"],dl["save_path"]
        deep_lng(X=df,y=df,test_size=test_size,ngram_range=ngram_range,fe_type=fe_type,hidden_layer_sizes=hidden_layer_sizes,activation=activation,solver=solver,learning_rate=learning_rate,max_iter=max_iter,ascend=ascend,save_path=save_path)
        
    #Similarity Metrics
    #Cosine Similarity
    cs= data["UnsupervisedLearning"]["cosine_similarity"]
    cosinesimilarity = cs["enable"]
    if cosinesimilarity:
        threshold,total_rows,base_row,ngram_range,fe_type,ascending= cs["threshold"],cs["total_rows"],cs["base_row"],cs["ngram_range"],cs["fe_type"],cs["ascending"]        
        cosinesimilarity(column=df,threshold=threshold,total_rows=total_rows,base_row=base_row,ngram_range=ngram_range,fe_type=fe_type,ascending=ascending)
        
    #Jaccard Similarity
    js= data["UnsupervisedLearning"]["jaccard_similarity"]
    jaccardsimilarity = js["enable"]
    if jaccardsimilarity:
        threshold,total_rows,base_row,ascending= js["threshold"],js["total_rows"],js["base_row"],js["ascending"]
        jaccardsimilarity(column=df,threshold=threshold,total_rows=total_rows,base_row=base_row,ascending=ascending)

In [65]:
json_path = "C:/Users/nchong/OneDrive - Intel Corporation/Documents/Debug Similarity Analytics and Bucketization Framework/General/"
main(json_path,out_path)

Files read: ['C:/Users/nchong/OneDrive - Intel Corporation/Documents/Debug Similarity Analytics and Bucketization Framework/General/Sample json output/HSD ES Raw Data/team1/(2021-08-25)1_firstSet_1.json', 'C:/Users/nchong/OneDrive - Intel Corporation/Documents/Debug Similarity Analytics and Bucketization Framework/General/Sample json output/HSD ES Raw Data/team1/(2021-08-25)3_secondSet_1.json', 'C:/Users/nchong/OneDrive - Intel Corporation/Documents/Debug Similarity Analytics and Bucketization Framework/General/Sample json output/HSD ES Raw Data/team1/(2021-10-11)3_secondSet_1.json']
Shape of df before manipulation: (2712, 15)
Shape of df after selecting columns: (2712, 2)
Number of null values in df:
 id             0
description    0
dtype: int64
Number of null values in df after NA imputation:
 id             0
description    0
dtype: int64
Number of duplicates in the df: 1808
Shape of df after manipulation: (904, 2)


,id,description,description_contract,description_contract_lower,description_contract_lower_tagrem,description_contract_lower_tagrem_puncrem,description_contract_lower_tagrem_puncrem_numrem,description_contract_lower_tagrem_puncrem_numrem_wsrem,description_contract_lower_tagrem_puncrem_numrem_wsrem_taxo
0,1308651592,Please provide a way to update GIO fields from...,Please provide a way to update GIO fields from...,please provide a way to update gio fields from...,please provide a way to update gio fields from...,please provide a way to update gio fields from...,please provide a way to update gio fields from...,please provide a way to update gio fields from...,please provide a way to update from git re...
1,1308671310,<p>Test suite execution finished before execut...,<p>Test suite execution finished before execut...,<p>test suite execution finished before execut...,test suite execution finished before executing...,test suite execution finished before executing...,test suite execution finished before executing...,test suite execution finished before executing...,test suite execution finished before executing...
2,1308673361,<p>I am trying to clone defects from another t...,<p>I am trying to clone defects from another t...,<p>i am trying to clone defects from another t...,i am trying to clone defects from another test...,i am trying to clone defects from another test...,i am trying to clone defects from another test...,i am trying to clone defects from another test...,i am trying to clone defects from another cy...
3,1507656633,Retest some function again.,Retest some function again.,retest some function again.,retest some function again.,retest some function again,retest some function again,retest some function again,retest some function again
4,1507656638,enter the support needed at here ...,enter the support needed at here ...,enter the support needed at here ...,enter the support needed at here ...,enter the support needed at here,enter the support needed at here,enter the support needed at here,enter the support needed at here
...,...,...,...,...,...,...,...,...,...
899,22012641037,"<div><span style=""font-size: 12.18px;"">Hello,&...","<div><span style=""font-size: 12.18px;"">Hello,&...","<div><span style=""font-size: 12.18px;"">hello,&...","hello, please import time global domain: time ...",hello please import time global domain time ...,hello please import time global domain time ...,hello please import time global domain time kp...,hello please import time global domain time kp...
900,22012645565,"<p>Hi Gio Team,</p><p><br /></p><p>Thank you f...","<p>Hi Gio Team,</p><p><br /></p><p>Thank you f...","<p>hi gio team,</p><p><br /></p><p>thank you f...","hi gio team, thank you for providing kpi_metri...",hi gio team thank you for providing kpi metri...,hi gio team thank you for providing kpi metri...,hi gio team thank you for providing kpi metric...,hi team thank you for providing kpi metric f...
901,22012704243,<div>The schedule test suite allow for the use...,<div>The schedule test suite allow for the use...,<div>the schedule test suite allow for the use...,the schedule test suite allow for the user to ...,the schedule test suite allow for the user to ...,the schedule test suite allow for the user to ...,the schedule test suite allow for the user to ...,the schedule suite allow for the user to clo...
902,22012765885,"<p>Hi Gio Team,</p><p><br /></p><p>Thank you f...","<p>Hi Gio Team,</p><p><br /></p><p>Thank you f...","<p>hi gio team,</p><p><br /></p><p>thank you f...","hi gio team, thank you for providing kpi featu...",hi gio team thank you for providing kpi featu...,hi gio team thank you for providing kpi featu...,hi gio team thank you for providing kpi featur...,hi team thank you for providing kpi feature ...


For n_clusters = 2 The silhouette_score is : 0.017
For n_clusters = 3 The silhouette_score is : 0.013
For n_clusters = 4 The silhouette_score is : 0.013
For n_clusters = 5 The silhouette_score is : 0.011
For n_clusters = 6 The silhouette_score is : 0.011
For n_clusters = 7 The silhouette_score is : 0.014
For n_clusters = 8 The silhouette_score is : 0.015
For n_clusters = 9 The silhouette_score is : 0.015
For n_clusters = 10 The silhouette_score is : 0.016

The optimal number of clusters selected is 2 with silhouette_score of 0.017 

Top 10 terms per cluster:
Cluster 0:
['project', 'dng', 'to', 'program', 'sve', 'requirement', 'create', 'yocto', 'link', 'request']


Cluster 1:
['the', 'to', 'in', 'is', 'and', 'for', 'cycle', 'not', 'of', 'this']




,id,description,cluster
0,1308651592,Please provide a way to update GIO fields from...,1
1,1308671310,<p>Test suite execution finished before execut...,1
2,1308673361,<p>I am trying to clone defects from another t...,1
3,1507656633,Retest some function again.,1
4,1507656638,enter the support needed at here ...,1
...,...,...,...
899,22012641037,"<div><span style=""font-size: 12.18px;"">Hello,&...",0
900,22012645565,"<p>Hi Gio Team,</p><p><br /></p><p>Thank you f...",1
901,22012704243,<div>The schedule test suite allow for the use...,1
902,22012765885,"<p>Hi Gio Team,</p><p><br /></p><p>Thank you f...",1


### Functions

In [5]:
#data loading
def data_loading(path,start_date=None,stop_date=None):
    '''
    Load only files that follow agreed filename format, merge files as single dataframe.
    User can choose to 
    a) Load all json files following the agreed filename format
    b) Load only json files from specific dates by adding the start and stop dates (Note: Both start_date and
    stop_date must be used together)
    
    params:
    path [string]: path of the files, without filename
    
    start_date[None/string in YYYY-MM-DD format](optional,default is None): 
    User can choose to load files starting from start_date
    - None: no start_date is provided, all files are loaded
    - string in YYYY-MM-DD format: files starting from start_date will be loaded
    
    stop_date[None/string in YYYY-MM-DD format](optional,default is None): 
    User can choose to load files until stop_date
    - None: no stop_date is provided, all files are loaded
    - string in YYYY-MM-DD format: files until stop_date will be loaded
    '''
    from datetime import datetime,timedelta
    import pandas as pd
    import glob, os, json
    import re
    
    filenames = os.listdir(path)
    file_list=[]
    date_list = []
    df = pd.DataFrame()
    
    if start_date == None and stop_date == None :
        for file in filenames:
            # search agreed file format pattern in the filename

            pattern = r"^\(\d{4}-\d{2}-\d{1,2}\)\d+\_\D+\_\d+\.json$"

            match = re.search(pattern,file)
                
            #if match is found
            if match:
                pattern = os.path.join(path, file) #join path with file name
                file_list.append(pattern) #list of json files that follow the agreed filename
            
        print("Files read:",file_list)                   
        for file in file_list:
            with open(file) as f:
                #flatten json into pd dataframe
                json_data = pd.json_normalize(json.loads(f.read()))
                json_data = pd.DataFrame(json_data)
                #label which file each row is from 
                json_data['file'] = file.rsplit("/", 1)[-1]

            df = df.append(json_data)              
                
    else:
        #convert start and stop string to datetime
        start = datetime.strptime(start_date, "%Y-%m-%d").date()
        stop = datetime.strptime(stop_date, "%Y-%m-%d").date()
    
        #iterate from start to stop dates by day and store dates in list
        while start <= stop:
            date_list.append(start)
            start = start + timedelta(days=1)  # increase day one by one

        #convert datetime objects to string
        string_list =[d.strftime("%Y-%m-%d") for d in date_list]
#         print(string_list)
        
        for file in filenames: 
            
            # search agreed file format pattern in the filename
            for date in string_list: 
                pattern = r"\("+date+r"\)\d+\_\D+\_\d+\.json"
        
                match = re.search(pattern,file)
                
                #if match is found
                if match:
                    pattern = os.path.join(path, file) #join path with file name
                    file_list.append(pattern) #list of json files that follow the agreed filename

        print("Files read:",file_list)     
        for file in file_list:
            with open(file) as f:
                #flatten json into pd dataframe
                json_data = pd.json_normalize(json.loads(f.read()))
                json_data = pd.DataFrame(json_data)
                #label which file each row is from 
                json_data['file'] = file.rsplit("/", 1)[-1]

            df = df.append(json_data)

    return df

In [47]:
#data preprocessing
def df_manipulation(df,col_selection=None,keep=None,subset=None):
    """
    1) Column selection: Keep columns in dataframe
    2) Data impute: Impute NA rows with empty string
    3) Data duplication cleaning: Drop all duplicates or drop all duplicates except for the first/last occurrence
    
    params:
    df [dataframe]: input dataframe     
    col_selection [None/list]: - None [Default]: Keep all columns in dataframe 
                               - List: List of columns to keep in dataframe                      
                                 
    keep[None/string/False]: Choose to drop all duplicates or drop all duplicates except for the first/last occurrence
                      # - None[DEFAULT] : Drop duplicates except for the first occurrence. 
                      # - "last" : Drop duplicates except for the last occurrence. 
                      # - False : Drop all duplicates.                 
    subset[list/None]: Subset of columns for identifying duplicates, use None if no column to select
   
    """
    
    print("Shape of df before manipulation:",df.shape)

    #Column selection - user can select column(s) 
    if col_selection != None:
        df = df[col_selection]
    
    print("Shape of df after selecting columns:",df.shape)

    #---Data impute - user can impute or drop rows with NA,freq of null values before & after manipulation returned---#
    print("Number of null values in df:\n",df.isnull().sum())
  

    # impute NA values with empty string
    impute_value = ""
    df = df.fillna(impute_value)
    print("Number of null values in df after NA imputation:\n",df.isnull().sum())

    #---------Data duplication cleaning--------#
    print("Number of duplicates in the df:", df.duplicated().sum())

    #drop duplicates
    if keep==None:
        keep="first"
    df = df.drop_duplicates(subset=subset, keep=keep)

    print("Shape of df after manipulation:",df.shape)

    return df

def word_contractions(df):
    """
    Expand word contractions (i.e. "isn't" to "is not")
    params:
    df [dataframe]: input dataframe 
    """
    import contractions
    import pandas as pd
    df = df.applymap(lambda text: " ".join([contractions.fix(word) for word in text.split()]))
    df = df.add_suffix('_contract')
    
    return df

def lowercase(df):
    """
    Convert all characters to lower case
    param:
    df[dataframe]: input dataframe
    
    """
    import pandas as pd
    df = df.applymap(lambda s:s.lower() if type(s) == str else s)
    df = df.add_suffix('_lower')
        
    return df 

def remove_htmltag_url(df):
    """
    Remove html tag and url
    params:
    df [dataframe]: input dataframe 
    
    """
    from bs4 import BeautifulSoup
    #remove html tag
    df = df.applymap(lambda text:BeautifulSoup(text, 'html.parser').get_text(separator= " ",strip=True))
    #remove url
    df = df.replace('https?[://%]*\S+',' ', regex=True) 
    
    df = df.add_suffix('_tagrem')
    
    return df

def remove_irrchar_punc(df,char=None):
    """
    Remove irrelevant characters and punctuation. Optional: User can specify special characters to be removed in regex
    format.    
    params:    
    df [dataframe]: input dataframe 
    characters[string]: input regex of characters to be removed  
    
    """
    import re 
    if char != None:
        #Remove special characters given by user
        df = df.replace(char,' ',regex = True)
            
    # Remove utf-8 literals (i.e. \\xe2\\x80\\x8)
    df = df.replace(r'\\+x[\d\D][\d\D]',' ',regex = True)
        
    #Remove special characters and punctuation
    df = df.replace('[^\w\s]',' ',regex = True)
    df = df.replace(r'_',' ',regex = True)
    
    df = df.add_suffix('_puncrem')
    
    return df

def remove_num(df):
    """
    Remove numeric data
    params:
    df [dataframe]: input dataframe 
    
    """
    df=df.replace('\d+',' ', regex=True) 
    df = df.add_suffix('_numrem')
    
    return df 

def remove_multwhitespace(df):
    """
    Remove multiple white spaces
    params:
    df [dataframe]: input dataframe 
    
    """
    df = df.replace(' +',' ', regex=True)
    df = df.add_suffix('_wsrem')
    
    return df


def remove_stopwords(df,extra_sw=None,remove_sw=None):
    """
    Removes English stopwords. Optional: user can add own stopwords or remove words from English stopwords  
    params:
    df [dataframe]: input dataframe 
    extra_sw [list] (optional): list of words/phrase to be added to the stop words 
    remove_sw [list] (optional): list of words to be removed from the stop words 
    """
    import nltk
    from nltk.corpus import stopwords

    all_stopwords = stopwords.words('english')
    
    #default list of stopwords
    if extra_sw == None and remove_sw==None:
        all_stopwords = all_stopwords
        
    # add more stopwords
    elif remove_sw == None:
        all_stopwords.extend(extra_sw) #add to existing stop words list
        
    # remove stopwords from existing sw list
    elif extra_sw == None:
        all_stopwords = [e for e in all_stopwords if e not in remove_sw] #remove from existing stop words list
        
    # remove and add stopwords to existing sw list
    else:
        all_stopwords.extend(extra_sw) #add to existing stop words list
        all_stopwords = [e for e in all_stopwords if e not in remove_sw] #remove from existing stop words list
         
  
    for w in all_stopwords:
        pattern = r'\b'+w+r'\b'
        df = df.replace(pattern,' ', regex=True)
    
    df = df.add_suffix('_stoprem')
                   
    return df 

def remove_freqwords(df,n):
    """
    Remove n frequent words
    params:
    df [dataframe]: input dataframe 
    n [integer]: input number of frequent words to be removed
    """
    from collections import Counter
    cnt = Counter()
    for i in df:
    
        for text in df[i].values:
            for word in text.split():
                cnt[word] += 1
           
    #custom function to remove the frequent words             
    FREQWORDS = set([w for (w, wc) in cnt.most_common(n)])
    
    print("Frequent words that are removed:", set([(w, wc) for (w, wc) in cnt.most_common(n)]))
    df = df.applymap(lambda text: " ".join([word for word in str(text).split() if word not in FREQWORDS]))
    df = df.add_suffix('_freqrem')
    return df

def remove_rarewords(df,n):
    """
    Remove n rare words
    params:
    df [dataframe]: input dataframe 
    n [integer]: input number of rare words to be removed
    """
    from collections import Counter
    cnt = Counter()
    for i in df:
    
        for text in df[i].values:
            for word in text.split():
                cnt[word] += 1
           
    #custom function to remove the frequent words             
    RAREWORDS = set([w for (w, wc) in cnt.most_common()[:-n-1:-1]])
    
    print("Rare words that are removed:", set([(w,wc) for (w, wc) in cnt.most_common()[:-n-1:-1]]))
    df = df.applymap(lambda text: " ".join([word for word in str(text).split() if word not in RAREWORDS]))
    df = df.add_suffix('_rarerem')
    return df


def custom_taxo(df,remove_taxo,include_taxo):
    """
    User provides taxonomy to be removed or remained in the text
    params:
    df [dataframe]: input dataframe
    remove_taxo[list]: list of taxonomy to be removed from text
    include_taxo[list]: list of taxonomy to be maintained in text
    """
    import re
    
    def taxo(text,remove_taxo,include_taxo):
        for w in remove_taxo:
        #row without any item from include_taxo -> replace all remove_taxo items with empty string
            if all(phrase not in text for phrase in include_taxo): 
                pattern = r'\b'+w+r'\b'
                text = re.sub(pattern,' ', text) 
            #row with any item from include_taxo -> only replace remove_taxo item that is not in include_taxo
            else: 
                if all(w not in phrase for phrase in include_taxo):
                    pattern = r'\b'+w+r'\b'
                    text = re.sub(pattern,' ', text) 
        return text 
    
    df = df.applymap(lambda text: taxo(text,remove_taxo,include_taxo))    
    df = df.add_suffix('_taxo')
                
    return df    

def stem_words(df,stemmer_type):
    """
    Stemming words. Default option is Porter Stemmer, alternative option is Lancaster Stemmer 
    params:
    df[dataframe]: input dataframe
    stemmer_type[None/string]: input stemming method 
                                - None for Porter Stemmer
                                - "Lancaster" for Lancaster Stemmer 
    """
    import pandas as pd
    import nltk
    from nltk.stem import PorterStemmer
    from nltk.stem import LancasterStemmer
    
    if stemmer_type == None:
        stemmer = PorterStemmer()
    if stemmer_type == "Lancaster":
        stemmer=LancasterStemmer()
    df = df.applymap(lambda text: " ".join([stemmer.stem(word) for word in text.split()]))
    df = df.add_suffix('_stem')
    
    
    return df

def lemmatize_words(df,lemma_type):
    """
    Lemmatize words: Default option is WordNetLemmatizer, alternative option is Spacy 
    params:
    df[dataframe]: input dataframe
    lemma_type[None/string]: input lemmatization method
                            - None for WordNetLemmatizer
                            - "Spacy" for Spacy    
    """
    import pandas as pd
    import spacy
    import nltk
    from nltk.stem import WordNetLemmatizer
    
    if lemma_type == None:
        lemmatizer = WordNetLemmatizer()
        df = df.applymap(lambda text: " ".join([lemmatizer.lemmatize(word) for word in text.split()]))
        
    if lemma_type == "Spacy":
        nlp = spacy.load("en_core_web_sm")
        df = df.applymap(lambda text: " ".join([word.lemma_ for word in nlp(text)]))
        #convert to lower case as spacy will convert pronouns to upper case
        df = df.applymap(lambda s:s.lower() if type(s) == str else s) 
        
    df = df.add_suffix('_lemma')
        
    return df

def feature_extraction(column,ngram_range=None,ascending=None,fe_type=None):
    """
    Feature extraction methods - TF-IDF(default choice) or Bag of words
     
    params:
    column [series/DataFrame]: column selected for feature extraction 
                        - series: only one column is selected for feature extraction (e.g. df["title_clean"])
                        - DataFrame: more than one column is selected for feature extraction (e.g. df[["title_clean","desc_clean"]])
    ngram_range [tuple(min_n, max_n)]: The lower and upper boundary of the range of n-values for different n-grams to be extracted
                                       - [default] ngram_range of (1, 1) means only unigrams, 
                                       - ngram_range of (1, 2) means unigrams and bigrams, 
                                       - ngram_range of (2, 2) means only bigram
    ascending [True/False/None]: - [default] None (words arranged in alphabetical order)
                                 - True(words arranged in ascending order of sum), 
                                 - False(words arranged in descending order of sum)                               
    fe_type[string/None]: Feature extraction type: Choose "bagofwords" for bow or None for default tfidf method
    
    """
    from sklearn.feature_extraction.text import CountVectorizer
    from sklearn.feature_extraction.text import TfidfVectorizer
    
    if type(column) == pd.DataFrame: #concat the columns into one string if there is more than one column 
        column = column.apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
                    
    if ngram_range == None: #set ngram range as unigram by default
        ngram_range=(1,1)
        
    if fe_type == "bagofwords":
        vec_type = CountVectorizer(ngram_range=ngram_range, analyzer='word')
        vectorized = vec_type.fit_transform(column)
        df = pd.DataFrame(vectorized.toarray(), columns=vec_type.get_feature_names())
        df.loc['sum'] = df.sum(axis=0).astype(int)

    if fe_type == None: #tfidf
        vec_type = TfidfVectorizer(ngram_range=ngram_range, analyzer='word')
        vectorized = vec_type.fit_transform(column)
        df = pd.DataFrame(vectorized.toarray(), columns=vec_type.get_feature_names())
        df.loc['sum'] = df.sum(axis=0)
    
    if ascending != None:
            
        df = df.sort_values(by ='sum', axis = 1,ascending=ascending)
    
    
    return df,vec_type,vectorized

In [28]:
#unsupervised learning

#K-means Clustering
import pandas as pd
def kmeans_clustering(column,top_n_terms,ngram_range=None,fe_type=None,n_clusters=None,max_n_clusters=None):
    """
    K- means clustering for unsupervised learning. User can choose either options:
    (1) provide the number of clusters or
    (2) provide the max number of clusters for kmeans to iterate through, the optimal number of clusters with highest 
    silhouette score will be chosen. Min number of clusters is fixed as 2
    
    params:
    column [series/DataFrame]: column(s) selected for clustering 
                        - series: only one column is selected for clustering (e.g. df["title_clean"])
                        - DataFrame: more than one column is selected for clustering (e.g. df[["title_clean","desc_clean"]])
    top_n_terms[int]: the top n terms in each cluster to be printed out
    ngram_range [tuple(min_n, max_n)]: The lower and upper boundary of the range of n-values for different n-grams to be extracted
                                   - [default] ngram_range of (1, 1) means only unigrams, 
                                   - ngram_range of (1, 2) means unigrams and bigrams, 
                                   - ngram_range of (2, 2) means only bigram
    fe_type[string/None]: Feature extraction type: Choose "bagofwords" for bow or None for default tfidf method
    n_clusters[None/int]: number of clusters. Choose None for option (2)  
    max_n_clusters[None/int]: max number of clusters. Choose None for option (1)  
    """   
    from sklearn.cluster import KMeans
    from sklearn import metrics

    silhouette_avg_list = []
    n_clusters_list = []
    dicts = {}
    
    #call feature extraction function    
    ascending = None 
    X = feature_extraction(column,ngram_range,ascending,fe_type)[0]
    X = X.drop(index='sum')
    vec_type = feature_extraction(column,ngram_range,ascending,fe_type)[1]

    #user provides the number of clusters        
    if n_clusters != None:
        model = KMeans(n_clusters = n_clusters, random_state=42)
        model.fit_predict(X)
        labels = model.labels_

        silhouette_score = metrics.silhouette_score(X, labels,random_state=42)
        print("Silhouette score for",n_clusters,"clusters is",round(silhouette_score,3))
        
            
    #user provides the maximum number of clusters 
    if max_n_clusters != None:
        for n_clusters in range(2,max_n_clusters+1): 

            model = KMeans(n_clusters = n_clusters, random_state=42)
            model.fit_predict(X)
            labels = model.labels_

            silhouette_avg = metrics.silhouette_score(X, labels,random_state=42)
            print("For n_clusters =", n_clusters,"The silhouette_score is :", round(silhouette_avg,3))

            silhouette_avg_list.append(silhouette_avg)
            n_clusters_list.append(n_clusters)


        for i in range(len(n_clusters_list)):
            dicts[n_clusters_list[i]] = silhouette_avg_list[i]

        n_clusters_max = max(dicts,key=dicts.get)
        silhouette_avg_max = max(dicts.values())

        model = KMeans(n_clusters = n_clusters_max, random_state=42)
        model.fit_predict(X)
        labels = model.labels_
        n_clusters = n_clusters_max
        print("\nThe optimal number of clusters selected is",n_clusters_max,"with silhouette_score of",round(silhouette_avg_max,3),"\n") 
        
    print("Top",top_n_terms,"terms per cluster:")
    order_centroids = model.cluster_centers_.argsort()[:, ::-1] #sort by descending order
    terms = vec_type.get_feature_names()
    for i in range(n_clusters):
        print("Cluster %d:" % i)
        print(['%s' % terms[ind] for ind in order_centroids[i, :top_n_terms]]) #top n terms in each cluster
        print("\n")
   
               
    return labels

#Latent Dirichlet Allocation
def lda(column,n_components,top_n_terms,ngram_range=None):
    """
    LDA for unsupervised learning. Bag of words is selected for feature extraction
    params:
    column [series/DataFrame]: column(s) selected for lda
                        - series: only one column is selected for lda (e.g. df["title_clean"])
                        - DataFrame: more than one column is selected for lda (e.g. df[["title_clean","desc_clean"]])
    n_components[int]: the number of topics/clusters used in the lda_model
    top_n_terms[int]: the top n terms in each topic/cluster to be printed out
    ngram_range [tuple(min_n, max_n)]: The lower and upper boundary of the range of n-values for different n-grams to be extracted
                                   - [default] ngram_range of (1, 1) means only unigrams, 
                                   - ngram_range of (1, 2) means unigrams and bigrams, 
                                   - ngram_range of (2, 2) means only bigram
    
    """
    from sklearn.decomposition import LatentDirichletAllocation
    
    #feature extraction
    ascending = None
    fe_type = "bagofwords"
    vec_type = feature_extraction(column,ngram_range,ascending,fe_type)[1]
    vectorized = feature_extraction(column,ngram_range,ascending,fe_type)[2]

    # Create object for the LDA class 
    lda_model = LatentDirichletAllocation(n_components, random_state = 42)  
    lda_model.fit(vectorized)
    
    # Components_ gives us our topic distribution 
    topic_words = lda_model.components_

    # Top n words for a topic

    for i,topic in enumerate(topic_words):
        print(f"The top {top_n_terms} words for topic #{i}")
        print([vec_type.get_feature_names()[index] for index in topic.argsort()[-top_n_terms:]])
        print("\n")
        
    topic_results = lda_model.transform(vectorized) #probabilities of doc belonging to particular topic
    
    
    return topic_results.argmax(axis=1)

#Non-negative Matrix Factorization 

def nmf(column,n_components,top_n_terms,fe_type,ngram_range=None):
    """
    Non-negative matrix factorization for unsupervised learning.
    params:
    column [series/DataFrame]: column(s) selected for NMF 
                        - series: only one column is selected for NMF (e.g. df["title_clean"])
                        - DataFrame: more than one column is selected for NMF (e.g. df[["title_clean","desc_clean"]])
    n_components[int]: the number of topics/clusters used in NMF
    top_n_terms[int]: the top n terms in each topic/cluster to be printed out
    fe_type[string/None]: Feature extraction type: Choose "bagofwords" for bow or None for default tfidf method
    ngram_range [tuple(min_n, max_n)]: The lower and upper boundary of the range of n-values for different n-grams to be extracted
                                   - [default] ngram_range of (1, 1) means only unigrams, 
                                   - ngram_range of (1, 2) means unigrams and bigrams, 
                                   - ngram_range of (2, 2) means only bigram
    """
    from sklearn.decomposition import NMF
    
    #feature extraction
    ngram_range = None
    ascending = None
    vec_type = feature_extraction(column,ngram_range,ascending,fe_type)[1]
    vectorized = feature_extraction(column,ngram_range,ascending,fe_type)[2]

    # Create object for the NMF class 
    nmf_model = NMF(n_components,random_state=42)
    nmf_model.fit(vectorized)
    
    # Components_ gives us our topic distribution 
    topic_words = nmf_model.components_

    # Top n words for a topic

    for i,topic in enumerate(topic_words):
        print(f"The top {top_n_terms} words for topic #{i}")
        print([vec_type.get_feature_names()[index] for index in topic.argsort()[-top_n_terms:]])
        print("\n")
        
    topic_results = nmf_model.transform(vectorized) 
    
    return topic_results.argmax(axis=1)

In [ ]:
#supervised learning

#Machine Learning
from sklearn.model_selection import train_test_split
from sklearn import metrics
import joblib
import numpy as np
def supervised_lng(X,y,test_size,ngram_range=None,fe_type=None,model_type=None,ascend=None,save_path=None):

    """
    Consists of 3 supervised machine learning methods: RandomForest (Default), Naive Bayes(optional, SVM (optional)
    
    X[series/DataFrame]: column(s) of text for supervised learning
                        - series: only one column is selected (e.g. df["title_clean"])
                        - DataFrame: more than one column is selected(e.g. df[["title_clean","desc_clean"]])
    y[series]: target 
    test_size[float/int]: If float, should be between 0.0 and 1.0 and represent the proportion of the dataset to include in the test split. 
                          If int, represents the absolute number of test samples.
    ngram_range [tuple(min_n, max_n)]: The lower and upper boundary of the range of n-values for different n-grams to be extracted
                                       -[DEFAULT] ngram_range of (1, 1) means only unigrams, 
                                       - ngram_range of (1, 2) means unigrams and bigrams, 
                                       - ngram_range of (2, 2) means only bigram
    fe_type[None/string]: Feature extraction type: Choose "bagofwords" or None for default tfidf method
    model_type[None/string]: Choose ML algorithm 
                            - None (Default algorithm is Random Forest)
                            - 'NB'(To choose Naive Bayes as ML algorithm), 
                            - 'SVM'(To choose Support Vector Machine as ML algorithm)
    ascend[True/False/None]:  - None (Default: Confusion matrix is arranged in alphabetical order)
                              - True(Confusion matrix arranged in ascending order of accuracy % per label), 
                              - False(Confusion matrix arranged in descending order of accuracy % per label)  
    save_path[None/string]: Path to save model
                            - None (Default - Model is not saved)
                            - String (Model is saved as model.joblib in the save_path specified as a string)
        
    """
    from sklearn.naive_bayes import MultinomialNB
    from sklearn.ensemble import RandomForestClassifier
    from sklearn import svm
    
    #TRAIN-TEST SPLIT
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_size, random_state = 42)
    print("Train-test split completed with",(1-test_size)*100,"-",test_size*100,"split in train-test")
    print("Shape of X_train is:", X_train.shape)
    print("Shape of X_test is:",X_test.shape)
    print("Shape of y_train is:",y_train.shape)
    print("Shape of y_test is:",y_test.shape)
    
    if type(X_train) == pd.DataFrame: #concat the columns into one string if there is more than one column 
        X_train = X_train.apply(lambda row: ' '.join(row.values.astype(str)), axis=1)         
#         display(X_train)
    if type(X_test) == pd.DataFrame: #concat the columns into one string if there is more than one column 
        X_test = X_test.apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
#         display(X_test)
    
    #FEATURE EXTRACTION
    column = X_train       
    ascending = None
    #fit_transform X_train
    X_train = feature_extraction(column,ngram_range,ascending,fe_type)[2]
    #only transform X_test
    vec_type = feature_extraction(column,ngram_range,ascending,fe_type)[1]
    X_test = vec_type.transform(X_test)
    
    
    print("Shape of X_train after feature extraction:",X_train.shape)
    print("Shape of X_test after feature extraction:",X_test.shape)
    
    #MODEL BUILDING
    if model_type == None:
        #random forest is chosen by default
        model = RandomForestClassifier(random_state = 42)
    
    if model_type == "NB":
        model = MultinomialNB()
                   
    if model_type == "SVM":
        model = svm.SVC(random_state = 42)
    
    model.fit(X_train, y_train) 
    
    #MODEL SAVING
    if save_path != None:
        joblib.dump(model, path + "model.joblib")
        print("Model saved!")

    # predicting test set results
    y_pred = model.predict(X_test)

    # MODEL EVALUATION  
   
    # print('Overall accuracy achieved is ' + str(round(metrics.accuracy_score(y_test, y_pred)*100,2)) + "%")
    # print("Classification report:\n",metrics.classification_report(y_test, y_pred,zero_division=0))
    
    #overall accuracy
    overall_acc = round(metrics.accuracy_score(y_test, y_pred)*100,2)
    overall_acc = {'Overall Acc %':overall_acc}
    overall_acc = pd.DataFrame([overall_acc])
    overall_acc.to_csv(save_path+"Overall_Accuracy.csv")

    #classification report
    report = metrics.classification_report(y_test, y_pred,zero_division=0,output_dict=True)
    report = pd.DataFrame(report).transpose()
    report.to_csv(save_path+"Classification_Report.csv")

    #confusion matrix with accuracies for each label
    class_accuracies = []

    for class_ in y_test.sort_values(ascending= True).unique():
        class_acc = round(np.mean(y_pred[y_test == class_] == class_)*100,2)
        class_accuracies.append(class_acc)
    class_acc = pd.DataFrame(class_accuracies,index=y_test.sort_values(ascending= True).unique(),columns= ["Accuracy %"])

    cf_matrix = pd.DataFrame(
        metrics.confusion_matrix(y_test, y_pred, labels= y_test.sort_values(ascending= True).unique()), 
        index=y_test.sort_values(ascending= True).unique(), 
        columns=y_test.sort_values(ascending= True).unique()
    )
    
    if ascend == None:
        cf_matrix = pd.concat([cf_matrix,class_acc],axis=1)
    else:
        cf_matrix = pd.concat([cf_matrix,class_acc],axis=1).sort_values(by=['Accuracy %'], ascending=ascend)
          
    cf_matrix.to_csv(out_path+"Confusion_Matrix.csv",index=False)     

#Deep Learning 

def deep_lng(X,y,test_size,ngram_range,fe_type,hidden_layer_sizes=None,activation=None,solver=None,learning_rate=None,max_iter=None,ascend=None,save_path=None):
    """
     Deep learning method: MultiLayer Perceptron

    X[series/DataFrame]: column(s) of text for deep learning
                        - series: only one column is selected (e.g. df["title_clean"])
                        - DataFrame: more than one column is selected(e.g. df[["title_clean","desc_clean"]])   
    y[series]: target
    test_size[float/int]: If float, should be between 0.0 and 1.0 and represent the proportion of the dataset to include in the test split. 
                          If int, represents the absolute number of test samples.
    ngram_range [tuple(min_n, max_n)]: The lower and upper boundary of the range of n-values for different n-grams to be extracted
                                       - ngram_range of (1, 1) means only unigrams, 
                                       - ngram_range of (1, 2) means unigrams and bigrams, 
                                       - ngram_range of (2, 2) means only bigram
    fe_type[string]: Feature extraction type: Choose "bagofwords" or "tfidf" method
    hidden_layer_sizes[tuple],default = (100): To set the number of layers and the number of nodes.
                                               Each element in the tuple represents the number of nodes,
                                               length of tuple denotes the total number of hidden layers in the network
    activation["identity", "logistic", "tanh","relu"], default="relu": Activation function for the hidden layer.
    solver["lbfgs", "sgd", "adam"], default="adam": The solver for weight optimization.
    learning_rate["constant", "invscaling", "adaptive"], default="constant": Learning rate schedule for weight updates
    max_iter[int], default=200: Maximum number of iterations. The solver iterates until convergence or this number of iterations.
    ascend [True/False/None]: - None (Default: Confusion matrix is arranged in alphabetical order)
                                 - True(Confusion matrix arranged in ascending order of accuracy % per label), 
                                 - False(Confusion matrix arranged in descending order of accuracy % per label)                            
    save_path[None/string]: Path to save model
                            - None (Default - Model is not saved)
                            - String (Model is saved as model.joblib in the save_path specified as a string)    
    """
    from sklearn.neural_network import MLPClassifier
    
    #train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_size, random_state = 42)
    print("Train-test split completed with",(1-test_size)*100,"-",test_size*100,"split in train-test")
    print("Shape of X_train is:", X_train.shape)
    print("Shape of X_test is:",X_test.shape)
    print("Shape of y_train is:",y_train.shape)
    print("Shape of y_test is:",y_test.shape)
    
    if type(X_train) == pd.DataFrame: #concat the columns into one string if there is more than one column 
        X_train = X_train.apply(lambda row: ' '.join(row.values.astype(str)), axis=1)         
        
    if type(X_test) == pd.DataFrame: #concat the columns into one string if there is more than one column 
        X_test = X_test.apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
        
    #FEATURE EXTRACTION
    column = X_train
    ascending = None
    #fit_transform X_train
    X_train = feature_extraction(column,ngram_range,ascending,fe_type)[2]
    #only transform X_test
    vec_type = feature_extraction(column,ngram_range,ascending,fe_type)[1]
    X_test = vec_type.transform(X_test)
    print("Shape of X_train after feature extraction:",X_train.shape)
    print("Shape of X_test after feature extraction:",X_test.shape)
    
    #MODEL BUILDING
    #default hypermarameters
    if hidden_layer_sizes == None:
        hidden_layer_sizes = (100)
    if activation == None:
        activation = "relu"
    if solver == None:
        solver = "adam"
    if learning_rate == None:
        learning_rate = "constant"
    if max_iter == None:
        max_iter = 200
    
    print("Hidden layer sizes: ", hidden_layer_sizes,", Activation: ",activation,", Solver: ",solver,", Learning rate: ",learning_rate,", Max iteration: ",max_iter)
    
    model = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes, activation=activation, solver=solver, max_iter=max_iter,verbose = False,random_state=42)
    model.fit(X_train,y_train)
    
    
    #MODEL SAVING
    if save_path != None:
        joblib.dump(model, path + "mlpmodel.joblib")
        print("Model saved!")

    # predicting test set results
    y_pred = model.predict(X_test)

    # MODEL EVALUATION  
   
    # print('Overall accuracy achieved is ' + str(round(metrics.accuracy_score(y_test, y_pred)*100,2)) + "%")
    # print("Classification report:\n",metrics.classification_report(y_test, y_pred,zero_division=0))
    
    #overall accuracy
    overall_acc = round(metrics.accuracy_score(y_test, y_pred)*100,2)
    overall_acc = {'Overall Acc %':overall_acc}
    overall_acc = pd.DataFrame([overall_acc])
    overall_acc.to_csv(save_path+"Overall_Accuracy.csv")

    #classification report
    report = metrics.classification_report(y_test, y_pred,zero_division=0,output_dict=True)
    report = pd.DataFrame(report).transpose()
    report.to_csv(save_path+"Classification_Report.csv")

    #confusion matrix with accuracies for each label
    class_accuracies = []

    for class_ in y_test.sort_values(ascending= True).unique():
        class_acc = round(np.mean(y_pred[y_test == class_] == class_)*100,2)
        class_accuracies.append(class_acc)
    class_acc = pd.DataFrame(class_accuracies,index=y_test.sort_values(ascending= True).unique(),columns= ["Accuracy %"])

    cf_matrix = pd.DataFrame(
        metrics.confusion_matrix(y_test, y_pred, labels= y_test.sort_values(ascending= True).unique()), 
        index=y_test.sort_values(ascending= True).unique(), 
        columns=y_test.sort_values(ascending= True).unique()
    )
    
    if ascend == None:
        cf_matrix = pd.concat([cf_matrix,class_acc],axis=1)
    else:
        cf_matrix = pd.concat([cf_matrix,class_acc],axis=1).sort_values(by=['Accuracy %'], ascending=ascend)
          
    cf_matrix.to_csv(out_path+"Confusion_Matrix.csv",index=False)   
    

In [ ]:
#similarity metrics

#Cosine Similarity 

def cosinesimilarity(column,threshold=None,total_rows = None,base_row=None,ngram_range=None,fe_type=None,ascending=None):
    """
    Compute the cosine similarity between rows of texts. User can 
    a) fix number of rows for comparison, each row will be taken as base and compared with the rest
    b) fix one row as base, comparison will be done with all the other rows
    
    params:
    
    column[series/DataFrame]: column(s) of text for row wise similarity comparison
                        - series: only one column is selected (e.g. df["title_clean"])
                        - DataFrame: more than one column is selected(e.g. df[["title_clean","desc_clean"]])  
    threshold[None/float]: cut off value for the cosine similarity, only texts with values above or equal to threshold
                           will be printed
                        - None: Default threhold is 0.5
                        - float: any value between 0 and 1 
    total_rows[None/int]: Number of rows for comparison, choose None for option b 
    base_row[None/int]: Row fixed as base, choose None for option a 
    ngram_range [tuple(min_n, max_n)]: The lower and upper boundary of the range of n-values for different n-grams to be extracted
                                       - ngram_range of (1, 1) means only unigrams, 
                                       - ngram_range of (1, 2) means unigrams and bigrams, 
                                       - ngram_range of (2, 2) means only bigram
    fe_type[None/string]: Feature extraction type: Choose "bagofwords" or None for tfidf
    ascending [True/False/None]: - [default] None (words arranged in alphabetical order)
                                 - True(words arranged in ascending order of sum), 
                                 - False(words arranged in descending order of sum)  
    
    """
    from sklearn.metrics.pairwise import cosine_similarity
    
    if type(column) == pd.DataFrame: #concat the columns into one string if there is more than one column 
        column = column.apply(lambda row: ' '.join(row.values.astype(str)), axis=1) 
                
    #feature extraction              
    X = feature_extraction(column=column,ngram_range=ngram_range,ascending=None,fe_type=fe_type)[0]
    X = X.drop(["sum"],axis = 0)
    
    #Get cosine similarity matrix
    similarity_matrix = pd.DataFrame(cosine_similarity(X))
    
    #threshold
    if threshold == None:
        threshold = 0.5
       
    if total_rows !=None: #fix number of rows for comparison, each row will be taken as base and compared with the rest
        for base in range(total_rows): 
            print ("")
            print ("Using index " + str(base) + " as base:") #fix one index as base
            
            #Create empty df
            column_names = ["Index", "Similarity Score", "Text"]
            results = pd.DataFrame(columns = column_names)
            
            for i in range(total_rows): #compare base with other index
                
                if similarity_matrix.iloc[base,i] >= threshold: #print if comparison shows that silarity metric is more than threshold
                    new_row = {'Index':i, 'Similarity Score':round(similarity_matrix.iloc[base,i],4), 'Text':column.iloc[i]}
                    #append row to the dataframe
                    results = results.append(new_row, ignore_index=True)
                    if ascending != None:            
                        results = results.sort_values(by ='Similarity Score', axis = 0,ascending=ascending)
                        
            display(results)
#             print(results['Similarity Score'].mean())
           

    if base_row !=None: #fix base_row index for comparison with all indexes
        print ("Using index " + str(base_row) + " as base:") #fix one index as base
        
        #Create empty df
        column_names = ["Index", "Similarity Score", "Text"]
        results = pd.DataFrame(columns = column_names)
        
        for i in range(len(column)): #compare base_row with other index
            if similarity_matrix.iloc[base_row,i] >= threshold: #print if comparison shows that silarity metric is more than threshold
                new_row = {'Index':i, 'Similarity Score':round(similarity_matrix.iloc[base_row,i],4), 'Text':column.iloc[i]}
                #append row to the dataframe
                results = results.append(new_row, ignore_index=True)
                if ascending != None:            
                    results = results.sort_values(by ='Similarity Score', axis = 0,ascending=ascending)  
                    
        results.to_csv(out_path+"Cosine_Similarity.csv",index=False)  

#Jaccard Similarity 

def jaccard_similarity(column,threshold=None,total_rows = None,base_row=None,ascending=None):
    """
    Compute the jaccard similarity between texts. User can 
    a) fix number of rows for comparison, each row will be taken as base and compared with the rest
    b) fix one row as base, comparison will be done with all the other rows
    
    params:
    column[series/DataFrame]: column(s) of text for row wise similarity comparison
                        - series: only one column is selected (e.g. df["title_clean"])
                        - DataFrame: more than one column is selected(e.g. df[["title_clean","desc_clean"]]) 
    threshold[None/float]: cut off value for the jaccard similarity, only texts with values above or equal to threshold
                           will be printed
                        - None: Default threhold is 0.5
                        - float: any value between 0 and 1 
    total_rows[None/int]: Number of rows for comparison, choose None for option b 
    base_row[None/int]: Row fixed as base, choose None for option a 
    ascending [True/False/None]: - [default] None (words arranged in alphabetical order)
                                 - True(words arranged in ascending order of sum), 
                                 - False(words arranged in descending order of sum)  
    
    """     
            
    #jaccard score computation
    def get_jaccard_sim(str1, str2):        
        a = set(str1.split()) 
        b = set(str2.split())
        c = a.intersection(b)
        return float(len(c)) / (len(a) + len(b) - len(c))
    
    if type(column) == pd.DataFrame: #concat the columns into one string if there is more than one column 
        column = column.apply(lambda row: ' '.join(row.values.astype(str)), axis=1) 
       
    #threshold
    if threshold == None:
        threshold = 0.5
        
    if total_rows !=None: #fix number of rows for comparison, each row will be taken as base and compared with the rest
        for base in range(total_rows): 
            print ("")
            print ("Using index " + str(base) + " as base:") #fix one index as base
            
            #Create empty df
            column_names = ["Index", "Similarity Score", "Text"]
            results = pd.DataFrame(columns = column_names)                   
            
            for i in range(total_rows): #compare base with other index
                jac_score =  round(get_jaccard_sim(column.iloc[base],column.iloc[i]),4)
                if jac_score > threshold: #print if comparison shows that silarity metric is more than threshold
                    new_row = {'Index':i, 'Similarity Score':jac_score, 'Text':column.iloc[i]}
                    #append row to the dataframe
                    results = results.append(new_row, ignore_index=True)
                if ascending != None:            
                    results = results.sort_values(by ='Similarity Score', axis = 0,ascending=ascending)  
                    
            display(results) 
        
    if base_row != None: #fix base_row index for comparison with all indexes
       
        print ("Using index " + str(base_row) + " as base row:") #fix one index as base_row
        #Create empty df
        column_names = ["Index", "Similarity Score", "Text"]
        results = pd.DataFrame(columns = column_names)                   
            
        for i in range(len(column)): #compare base_row with other index
            jac_score = round(get_jaccard_sim(column.iloc[base_row],column.iloc[i]),4)
            if jac_score >= threshold: #print if comparison shows that silarity metric is more than threshold
                new_row = {'Index':i, 'Similarity Score':jac_score, 'Text':column.iloc[i]}
                #append row to the dataframe
                results = results.append(new_row, ignore_index=True)
            if ascending != None:            
                results = results.sort_values(by ='Similarity Score', axis = 0,ascending=ascending)  

        results.to_csv(out_path+"Jaccard_Similarity.csv",index=False)  

In [ ]:
# config 1: DL -> DP -> Supervised learning
# config 2: DL -> DP -> Unsupervised learning
# config 3: DL -> DP -> Similarity metrics 

In [ ]:
#read config file
# import json
# json_path = "C:/Users/nchong/OneDrive - Intel Corporation/Documents/Debug Similarity Analytics and Bucketization Framework/General/"

# with open(json_path+'config.json') as config_file:
#     data = json.load(config_file)

# path = data["DataLoading"]['path']
# start_date = data["DataLoading"]['start_date']
# stop_date = data["DataLoading"]['stop_date']
# print(path)
# print(start_date)
# print(stop_date)


In [ ]:
#     method #1
#     ["title","Desc","comment"]
    
#     wordcont = data["df_manipulation"]["word_contractions"]
#     if wordcont:
#         df["title_cont"] = [word_contractions(text) for text in df["title"]]
#         df["desc_cont"] = [word_contractions(text) for text in df["desc"]]
#         df["comment_cont"] = [word_contractions(text) for text in df["comment"]
    
#     method #2
#     df (title,desc,comments) -> df(title,desc,comments) + df (title_cont,desc_cont,comments_cont)
#     df (title,desc,comments) -> concat df(title,desc,comments) [df1]
#                               columnbind(df,df1)

In [ ]:

# config = {
#     "DataLoading": {"path": path, "start_date": None,"stop_date":None},
#     "df_manipulation": {"col_selection":["id","description"],"keep": None,"subset":None},    
#     "word_contractions": {"enable": True},
#     "lowercase": {"enable": True},
#     "remove_htmltag_url": {"enable":True},
#     "remove_irrchar_punc":{"enable":True,"char":None},
#     "remove_num":{"enable":True},
#     "remove_multwhitespace":{"enable":True},
#     "remove_stopwords":{"enable":True,"extra_sw":None,"remove_sw": None},
#     "remove_freqwords":{"enable":True,"n":10},
#     "remove_rarewords":{"enable":True,"n":10},
#     "stem_words":{"enable":False,"stemmer_type":None},
#     "lemmatize_words":{"enable":True,"lemma_type":None}

# }